# f) Ngonye Falls Power and Energy Periods
Produces summary, agrregated results from the daily power and energy modelling.


## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| [Model Name]_pe_daily.csv | Notebook: e_power_energy | Daily power output  |


## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|
|    [Model Name]_pe_daily.csv ||
|    [Model Name]_pe_monthly.csv||
|    [Model Name]_pe_yearly.csv||
|    [Model Name]_pe_percentiles.csv||
|    [Model Name]_pe_monthly_day.csv||
|    [Model Name]_pe_daily_slim.csv||
|    [Model Name]_pe_weekly.csv||
|    [Model Name]_pe_weekly_slim.csv||
|    [Model Name]_pe_calmonthly.csv||
|    [Model Name]_pe_fdc.csv||
|    [Model Name]_pe_power_exceed.csv||
|    models.csv| |

## Libraries and Setup

In [1]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math
import statsmodels.tsa.stattools as statools

## Parameters

In [2]:
year = "2024"

In [3]:
input_data='./input_data/'
output_data='./output_data/'

if year != '':
    output_data += year + '/'

input_data, output_data

('./input_data/', './output_data/2024/')

Set the model name below. This is used to lookup model parameters from the models list file.

- Base Case: Base Case
- MW162: Capacity reduced to 162MW
- Headpond50cm: Headpond level increased by 50cm
- EWRAllCs: All river reaches at a C rather than B environmental assurance level
- Extend Head: Minimum head reduce from 10m to 9m
- Tailwater Rise 50cm: Tailwater curve raised by 50cm
- Tailwater Fall 50cm: Tailwater curve dropped by 50cm

In [40]:
model_name='Base Case'
dryrun=False #Dont write output files
output_separate_years=True

In [5]:
if year != '':
    model_file=input_data + year + "/models.csv"
else:
    model_file=input_data + "models.csv"

models=pd.read_csv(model_file).set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'
if out_prefix!='base':
    out_prefix="/scenarios/" + out_prefix

category_set_name=model['EWRCatSet']#'Recommendation 1'
headpond_lift=model['HeadpondLift']#0
plant_capacity=model['Capacity'] #Base 180MW nominal capacity. Only used to calculate 'nominal capacity factor'
units=4 #Design no of units
units_avialable=model['UnitsAvailable'] #Available no of units - to see what happens during testing/commissioning

rated_flow_unit=model['RatedDischarge'] #250
max_flow_unit=rated_flow_unit*model['MaximumLoadFactor'] #275
canal_capacity=max_flow_unit*4 #canal capacity 1100 cumec 
min_flow_unit=model['MinimumFlowUnit'] #50cumec minimum flow for a turbine
fixed_tailwater_level=model['FixedTailwater']#False #970 
head_minimum=model['MinimumHead'] #minimum head 10m

rated_power_unit=rated_flow_unit*model['RatedHead']*model['RatedTurbineEfficiency']*model['RatedGeneratorEfficiency']*9.81/1000

model

OutputPrefix                            base
Description                        Base Case
EWRCatSet                   Recommendation 1
TailwaterLift                            0.0
HeadpondLift                             0.0
Capacity                                 180
MinimumHead                               10
MinimumFlowUnit                           50
FixedTailwater                             0
UnitsAvailable                             4
RatedDischarge                           250
RatedHead                               21.7
RatedTurbineEfficiency                 0.943
RatedGeneratorEfficiency                0.98
MaximumLoadFactor                        1.1
OptimiseTurbines                           0
RatedPower                               NaN
Mean                                     NaN
P95                                      NaN
P90                                      NaN
P75                                      NaN
P50                                      NaN
P25       

## Load Daily Data

Load the daily time series which includes the exceedance values used for calculating daily EWRs.

In [12]:
daily_file = output_data + out_prefix + '_pe_daily.csv'
daily=pd.read_csv(daily_file) #ngonye_daily_WC1_1_base_pe
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)
daily_file, daily.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 36514 entries, 1924-10-01 to 2024-09-19
Data columns (total 83 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   LaggedDate                  36514 non-null  object 
 1   VicFalls                    36514 non-null  float64
 2   Conversion                  36514 non-null  float64
 3   Flow                        36514 non-null  float64
 4   Exceedance                  36514 non-null  float64
 5   Year                        36514 non-null  int64  
 6   Month                       36514 non-null  int64  
 7   Day                         36514 non-null  int64  
 8   MonthId                     36514 non-null  float64
 9   WaterYear                   36514 non-null  int64  
 10  WaterMonth                  36514 non-null  int64  
 11  WaterDay                    36514 non-null  int64  
 12  WaterWeek                   36514 non-null  int64  
 13  Volume        

('./output_data/2024/base_pe_daily.csv', None)

In [13]:
monthly=daily.groupby("MonthId").mean(numeric_only=True).drop(['Day','EWRBandNo','EWRRefExceedance','Energy'],axis=1)

if 'VicFalls' in monthly.columns:
    monthly=monthly.drop(['VicFalls','Conversion','Volume','WaterDay','WaterWeek','Volume'],axis=1)

monthly=monthly.join(daily.groupby("MonthId").agg(    
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),   
   Turbines_max=('Turbines', 'max'),    
   Turbines_min=('Turbines', 'min'),   
   Energy=('Energy', 'sum'),
   FlowAvailableForGeneration_min=('FlowAvailableForGeneration','min')
))
monthly['Days']=monthly.apply(lambda x: pd.Period(year=x['Year'],month=x['Month'],freq='D').days_in_month,axis=1)
monthly['CapFactor']=monthly['Energy']/(plant_capacity*24*monthly['Days'])

monthly

,Flow,Exceedance,Year,Month,WaterYear,WaterMonth,Flow_difference,Flow_difference_pct,EWRChannelA,EWRChannelC,...,PotentialPower3u,PotentialPower4u,Power_max,Power_min,Turbines_max,Turbines_min,Energy,FlowAvailableForGeneration_min,Days,CapFactor
MonthId,,,,,,,,,,,,,,,,,,,,,
1924.10,134.493603,0.998548,1924.0,10.0,1924.0,1.0,0.623161,0.020760,28.193548,4.322581,...,170.816235,224.367670,18.041268,0.000000,1,0,8509.130373,39.495189,31,0.063539
1924.11,153.220935,0.992267,1924.0,11.0,1924.0,2.0,2.525339,0.016032,23.000000,4.000000,...,170.889995,224.478598,31.064854,16.455300,1,1,15830.713484,80.058184,30,0.122151
1924.12,276.286727,0.832129,1924.0,12.0,1924.0,3.0,9.712352,0.031345,26.000000,4.000000,...,169.566658,222.505788,81.962655,29.917238,2,1,35577.350204,136.246123,31,0.265661
1925.01,717.515348,0.458355,1925.0,1.0,1924.0,4.0,45.368341,0.056403,37.129032,5.677419,...,159.852431,208.436215,193.336265,74.401324,4,2,90513.392200,349.232167,31,0.675877
1925.02,1217.500493,0.277321,1925.0,2.0,1924.0,5.0,23.949569,0.019245,49.000000,8.000000,...,148.658052,192.607864,192.236795,177.624315,4,4,124651.969090,964.839328,28,1.030522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024.05,827.433756,0.396839,2024.0,5.0,2023.0,8.0,4.909827,0.006014,42.000000,9.000000,...,157.370803,204.861977,142.536465,119.368082,3,3,99916.421377,601.927190,31,0.746090
2024.06,661.708649,0.478133,2024.0,6.0,2023.0,9.0,13.080206,0.020325,43.000000,9.666667,...,161.136799,210.246655,145.844815,65.619405,3,2,75034.469714,313.763172,30,0.578970
2024.07,399.628460,0.660484,2024.0,7.0,2023.0,10.0,4.160321,0.010306,42.000000,9.000000,...,166.995165,218.707511,70.229330,46.030151,2,1,42361.848274,210.793207,31,0.316322


In [14]:
yearly=daily.groupby("WaterYear").agg(
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),   
   PowerTurbine1_max=('PowerTurbine1', 'max'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
)
yearly['CapFactor']=yearly['Energy']/(plant_capacity*24*365)


In [15]:
yearly.tail(5)

,Flow_max,Flow_min,EWRProportion_max,EWRProportion_min,HeadTurbine1_max,HeadTurbine1_min,PowerTurbine1_max,LevelTailwater_max,LevelTailwater_min,Power_max,Power_min,Energy,CapFactor
WaterYear,,,,,,,,,,,,,
2019,4669.790989,128.064107,0.444243,0.054447,25.383720,11.157380,60.575492,976.905718,964.400695,191.214887,15.637731,813401.245143,0.515856
2020,3731.765568,260.019625,0.471475,0.050991,24.977198,12.951500,58.908372,975.111598,964.740078,192.304632,33.491793,920409.380670,0.583720
2021,3102.277326,264.119331,0.530562,0.070387,24.981104,14.302429,59.117550,973.760669,964.756477,191.858860,31.608322,898026.797604,0.569525
2022,2768.350384,228.804955,0.521057,0.061844,25.201014,15.108459,58.908372,972.954639,964.572830,190.419684,28.382294,917848.133839,0.582095
2023,875.472103,218.764823,0.459751,0.137703,25.270829,21.651606,59.780303,967.401888,964.512589,145.844815,27.958439,697726.681531,0.442495


In [16]:
yearly['Prop4Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==4,1,0))/365)
yearly['Prop3Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==3,1,0))/365)
yearly['Prop2Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==2,1,0))/365)
yearly['Prop1Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==1,1,0))/365)
yearly['Prop0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0))/365)
yearly['Days0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0)))

In [17]:
yearly['PropDays1uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']>rated_power_unit).groupby('WaterYear').count()
yearly['PropDays2uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower2u']].where(lambda x: x['PotentialPower2u']>rated_power_unit*2).groupby('WaterYear').count()
yearly['PropDays3uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower3u']].where(lambda x: x['PotentialPower3u']>rated_power_unit*3).groupby('WaterYear').count()
yearly['PropDays4uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower4u']].where(lambda x: x['PotentialPower4u']>yearly['Power_max'].max()).groupby('WaterYear').count()

In [18]:
daily.reset_index()[['Date','WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']<rated_power_unit).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

WaterYear
1924.0     58
1925.0     67
1926.0     54
1927.0     68
1928.0    107
         ... 
2017.0     40
2019.0     63
2020.0     37
2021.0     68
2022.0     37
Name: Date, Length: 89, dtype: int64

In [19]:
yearly['FstDay1uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']<rated_power_unit).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay1uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']<rated_power_unit).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

yearly['FstDay2uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower2u']].where(lambda x: x['PotentialPower2u']<rated_power_unit*2).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay2uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower2u']].where(lambda x: x['PotentialPower2u']<rated_power_unit*2).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

yearly['FstDay3uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower3u']].where(lambda x: x['PotentialPower3u']<rated_power_unit*3).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay3uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower3u']].where(lambda x: x['PotentialPower3u']<rated_power_unit*3).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

yearly['FstDay4uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower4u']].where(lambda x: x['PotentialPower4u']<yearly['Power_max'].max()).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay4uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower4u']].where(lambda x: x['PotentialPower4u']<yearly['Power_max'].max()).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)


## Power and energy exceedance

In [20]:
power_exceed=pd.DataFrame(index=np.arange(0,1.05,0.05))
power_exceed.index=power_exceed.index.rename('Exceedance')
power_exceed['Power']=np.percentile(daily['Power'],(1-power_exceed.index)*100)
power_exceed

,Power
Exceedance,
0.00,195.025499
0.05,183.509576
0.10,173.994442
0.15,164.304593
0.20,155.431298
0.25,147.311315
0.30,138.752753
0.35,129.586090
0.40,116.569765


In [21]:
edc=pd.DataFrame(index=np.arange(0,1.05,0.05))
edc['Energy']=np.percentile(yearly['Energy'],(1-edc.index)*100)

edc

,Energy
0.00,1.033636e+06
0.05,9.722217e+05
0.10,9.337445e+05
0.15,9.182323e+05
0.20,9.022031e+05
0.25,8.883655e+05
0.30,8.741188e+05
0.35,8.644780e+05
0.40,8.579910e+05
0.45,8.541644e+05


In [22]:
yearly['Exceedance']=pd.merge_asof(yearly.reset_index().sort_values('Energy'),edc.reset_index().sort_values('Energy'),left_on='Energy',right_on='Energy').set_index('WaterYear')['index']

## Calendar years
Jan-Dec rather than Oct-Sept

In [23]:
min_year=daily['Year'].min()
max_year=daily['Year'].max()


In [24]:
calyearly=daily.groupby("Year").agg(  
   Energy=('Energy', 'sum'),
).drop([min_year,max_year],axis=0)
calyearly['CapFactor']=calyearly['Energy']/(plant_capacity*24*365)
calyearly.head(5)

,Energy,CapFactor
Year,,
1925,866292.928693,0.549399
1926,789927.311837,0.500969
1927,865804.936625,0.549090
1928,840886.181297,0.533287
1929,765997.142957,0.485792


## Monthly Day
15th of every month

In [25]:
monthly_day=daily.loc[daily['Day']==15]

## Daily Slim
Slim has less columns and values are rounded so file size is reduced

In [26]:
daily_slim=daily.loc[:,
                     ['Year',
                      'Month',
                      'Day',
                      'WaterYear',
                      'Flow',
                      'EWRRefExceedance',
                      'EWRBandNo',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'Turbines',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',    
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion': 2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,    
                    })
daily_slim.tail(12)

,Year,Month,Day,WaterYear,Flow,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,EffTurbine1,EffTurbine2,EffTurbine3,EffTurbine4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy
Date,,,,,,,,,,,,,,,,,,,,,
2024-09-08,2024,9,8,2023,239,0.80,4,34,6,4,...,0.92,0.0,0.0,0.0,32.6,0.0,0.0,0.0,32.6,781
2024-09-09,2024,9,9,2023,237,0.80,4,34,6,4,...,0.91,0.0,0.0,0.0,31.8,0.0,0.0,0.0,31.8,763
2024-09-10,2024,9,10,2023,234,0.80,4,34,6,4,...,0.91,0.0,0.0,0.0,31.3,0.0,0.0,0.0,31.3,752
2024-09-11,2024,9,11,2023,233,0.82,4,34,6,4,...,0.91,0.0,0.0,0.0,31.0,0.0,0.0,0.0,31.0,743
2024-09-12,2024,9,12,2023,231,0.82,4,34,6,4,...,0.91,0.0,0.0,0.0,30.5,0.0,0.0,0.0,30.5,732
2024-09-13,2024,9,13,2023,231,0.82,4,34,6,4,...,0.91,0.0,0.0,0.0,30.5,0.0,0.0,0.0,30.5,732
2024-09-14,2024,9,14,2023,229,0.82,4,34,6,4,...,0.91,0.0,0.0,0.0,30.1,0.0,0.0,0.0,30.1,723
2024-09-15,2024,9,15,2023,229,0.82,4,34,6,4,...,0.91,0.0,0.0,0.0,30.1,0.0,0.0,0.0,30.1,723
2024-09-16,2024,9,16,2023,229,0.82,4,34,6,4,...,0.91,0.0,0.0,0.0,30.1,0.0,0.0,0.0,30.1,723


## Weekly & Weekly Slim
Slim has less columns and values are rounded so file size is reduced

In [27]:
weekly=daily.groupby(["WaterYear","WaterWeek"]).mean(numeric_only=True).drop(['Month','WaterMonth','Year','MonthId','Day','WaterDay','EWRBandNo','EWRRefExceedance','Turbines','Energy'],axis=1)

#if 'VicFalls' in weekly.columns:
weekly=weekly.drop(['VicFalls','Conversion','Volume'],axis=1)

weekly=weekly.join(daily.groupby(["WaterYear","WaterWeek"]).agg(    
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
))
weekly=weekly.join(daily[["WaterYear","WaterWeek","Flow"]].groupby(["WaterYear","WaterWeek"]).count().rename(columns={'Flow':'Count'}))
weekly['CapFactor']=weekly['Energy']/(plant_capacity*24*weekly['Count'])
weekly=weekly.join(daily.reset_index('Date')[["WaterYear","WaterWeek","Date"]].groupby(["WaterYear","WaterWeek"]).first().rename(columns={'Date':'StartDate'}))

weekly.tail(12)

Flow  Exceedance  Flow_difference  \
WaterYear WaterWeek                                            
2023      40         457.848666    0.599429         5.705035   
          41         416.332126    0.640714         4.997174   
          42         395.402507    0.663714         3.472375   
          43         368.140520    0.696000         3.911092   
          44         345.733677    0.721143         2.704928   
          45         326.756834    0.749571         2.566662   
          46         306.310245    0.778143         3.124553   
          47         284.970132    0.811143         3.345006   
          48         265.584372    0.843429         1.782468   
          49         250.395517    0.876286         3.404496   
          50         233.723249    0.902286         1.438647   
          51         225.245994    0.917000         2.084936   

                     Flow_difference_pct  EWRChannelA  EWRChannelC  \
WaterYear WaterWeek                                                  
2023      40                    0.012410    42.428571     9.142857   
          41                    0.011904    42.000000     9.000000   
          42                    0.008824    42.000000     9.000000   
          43                    0.010627    42.000000     9.000000   
          44                    0.007827    40.714286     8.142857   
          45                    0.007847    39.000000     7.000000   
          46                    0.010184    39.000000     7.000000   
          47                    0.011793    39.000000     7.000000   
          48                    0.006683    39.000000     7.000000   
          49                    0.013700    34.000000     6.000000   
          50                    0.006170    34.000000     6.000000   
          51                    0.009387    34.000000     6.000000   

                     EWRChannelD  EWRChannelE  EWRChannelFG    EWRTotal  ...  \
WaterYear WaterWeek                                                      ...   
2023      40            8.142857     6.428571     72.571429  138.714286  ...   
          41            8.000000     6.000000     70.000000  135.000000  ...   
          42            8.000000     6.000000     70.000000  135.000000  ...   
          43            8.000000     6.000000     70.000000  135.000000  ...   
          44            6.714286     5.142857     65.714286  126.428571  ...   
          45            5.000000     4.000000     60.000000  115.000000  ...   
          46            5.000000     4.000000     60.000000  115.000000  ...   
          47            5.000000     4.000000     60.000000  115.000000  ...   
          48            5.000000     4.000000     60.000000  115.000000  ...   
          49            4.000000     3.000000     44.000000   91.000000  ...   
          50            4.000000     3.000000     44.000000   91.000000  ...   
          51            4.000000     3.000000     44.000000   91.000000  ...   

                     HeadTurbine1_max  HeadTurbine1_min  LevelTailwater_max  \
WaterYear WaterWeek                                                           
2023      40                24.041648         23.869882          965.736434   
          41                24.226996         23.846832          965.566313   
          42                24.077511         23.890709          965.420403   
          43                24.320014         24.102737          965.305883   
          44                24.413430         24.341281          965.170667   
          45                24.567123         24.430159          965.078639   
          46                24.744218         24.602339          964.977492   
          47                24.904976         24.757162          964.883899   
          48                24.987496         24.928924          964.783269   
          49                25.098651         24.957533          964.728781   
          50                25.183989         25.098651          964.635560   
 

In [28]:
weekly_slim=weekly.loc[:,
                     [
                      'StartDate',
                      'Flow',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion':2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,
                    })

## Calendar Months

In [29]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly=calmonthly.join(monthly.groupby(["WaterMonth"]).agg(    
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_mean=('EWRProportion', 'mean'),
   EWRProportion_median=('EWRProportion', 'median'),
   EWRProportion_min=('EWRProportion', 'min'),
    
    EWRTotal_max=('EWRTotal', 'max'),
   EWRTotal_mean=('EWRTotal', 'mean'),
   EWRTotal_median=('EWRTotal', 'median'),
   EWRTotal_min=('EWRTotal', 'min'),   
    
   FlowCanal_max=('FlowCanal', 'max'),
   FlowCanal_mean=('FlowCanal', 'mean'),
   FlowCanal_median=('FlowCanal', 'median'),
   FlowCanal_min=('FlowCanal', 'min'),     
    
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_mean=('HeadTurbine1', 'mean'),
   HeadTurbine1_median=('HeadTurbine1', 'median'),
   HeadTurbine1_min=('HeadTurbine1', 'min'), 

   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_mean=('LevelTailwater', 'mean'),    
   LevelTailwater_median=('LevelTailwater', 'median'),    
   LevelTailwater_min=('LevelTailwater', 'min'),  
   LevelHeadpond_max=('LevelHeadpond', 'max'),
   LevelHeadpond_mean=('LevelHeadpond', 'mean'),    
   LevelHeadpond_median=('LevelHeadpond', 'median'),    
   LevelHeadpond_min=('LevelHeadpond', 'min'),  
   Turbines_mean=('Turbines', 'mean'),      
   Power_max=('Power', 'max'),
   Power_mean=('Power', 'mean'),
   Power_median=('Power', 'median'),
   Power_min=('Power', 'min'),    
   Energy_max=('Energy', 'max'),
   Energy_mean=('Energy', 'mean'),
   Energy_median=('Energy', 'median'),    
   Energy_min=('Energy', 'min'),
))
calmonthly['Energy_P95']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.05)
calmonthly['Energy_P90']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.10)
calmonthly['Energy_P75']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.25)
calmonthly['Energy_P25']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.75)
calmonthly['Energy_P10']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.90)
calmonthly['Energy_P5']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.95)

calmonthly['Power_P95']=daily[['WaterMonth','Power']].groupby('WaterMonth').quantile(0.05)
calmonthly['Power_P90']=daily[['WaterMonth','Power']].groupby('WaterMonth').quantile(0.10)

calmonthly['FlowAvailableForGeneration_P95']=monthly[['WaterMonth','FlowAvailableForGeneration']].groupby('WaterMonth').quantile(0.05)
calmonthly['FlowAvailableForGeneration_P50']=monthly[['WaterMonth','FlowAvailableForGeneration']].groupby('WaterMonth').quantile(0.5)

calmonthly['PotentialPower1u_P95']=monthly[['WaterMonth','PotentialPower1u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower1u_P50']=monthly[['WaterMonth','PotentialPower1u']].groupby('WaterMonth').quantile(0.5)
calmonthly['PotentialPower2u_P95']=monthly[['WaterMonth','PotentialPower2u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower2u_P50']=monthly[['WaterMonth','PotentialPower2u']].groupby('WaterMonth').quantile(0.5)
calmonthly['PotentialPower3u_P95']=monthly[['WaterMonth','PotentialPower3u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower3u_P50']=monthly[['WaterMonth','PotentialPower3u']].groupby('WaterMonth').quantile(0.5)
calmonthly['PotentialPower4u_P95']=monthly[['WaterMonth','PotentialPower4u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower4u_P50']=monthly[['WaterMonth','PotentialPower4u']].groupby('WaterMonth').quantile(0.5)


calmonthly['Energy_stdev']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').std()
calmonthly['Energy_coefvar']=calmonthly['Energy_stdev']/calmonthly['Energy_mean']

calmonthly

,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,PotentialPower1u_P95,PotentialPower1u_P50,PotentialPower2u_P95,PotentialPower2u_P50,PotentialPower3u_P95,PotentialPower3u_P50,PotentialPower4u_P95,PotentialPower4u_P50,Energy_stdev,Energy_coefvar
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1.0,Oct,10,0.493289,0.395903,0.399534,0.269664,120.0,93.734194,97.000000,52.0,...,56.813870,57.496015,113.106574,114.529408,168.257019,170.537797,220.560318,223.950380,6883.756576,0.293074
2.0,Nov,11,0.405300,0.333046,0.339519,0.210883,114.0,89.626667,93.000000,50.0,...,56.328330,57.268942,112.099236,114.054864,166.655351,169.774916,218.209197,222.811503,9063.109685,0.310262
3.0,Dec,12,0.285853,0.239748,0.241942,0.151971,126.0,100.371290,104.000000,59.0,...,54.283947,56.372208,107.889870,112.191266,160.040351,166.804103,208.676408,218.433073,15178.690753,0.286356
4.0,Jan,1,0.220992,0.172098,0.175889,0.096964,150.0,118.035484,120.000000,73.0,...,50.114853,54.667887,99.396020,108.676247,146.913513,161.265967,190.270558,210.419160,20094.599541,0.232588
5.0,Feb,2,0.184513,0.126249,0.135856,0.023904,197.0,139.093227,137.000000,85.0,...,37.358513,52.155200,73.631552,103.536139,107.653122,153.270340,136.537153,199.086104,17967.206051,0.179736
6.0,Mar,3,0.174545,0.095402,0.091399,0.040419,315.0,194.305161,193.258065,107.0,...,24.109111,42.484408,41.074040,83.944574,47.119083,123.268080,31.168967,157.667615,30547.192785,0.294403
7.0,Apr,4,0.186841,0.086423,0.083453,0.050048,364.0,231.410000,217.000000,125.0,...,26.370466,38.597681,51.621779,76.117206,74.575318,111.397736,70.958952,141.558395,28602.906345,0.294217
8.0,May,5,0.213956,0.130269,0.127393,0.065320,340.0,265.630968,276.000000,147.0,...,33.679016,43.056755,66.256528,85.099922,96.557313,125.027017,120.485237,160.071169,17276.307038,0.151620
9.0,Jun,6,0.367320,0.195159,0.190663,0.108489,293.0,213.166667,210.000000,119.0,...,44.465818,50.783768,87.945300,100.746113,129.361485,148.968499,166.000358,193.077750,22953.167588,0.214589


In [30]:
calmonthly[['Power_P90','Power_P95']]

,Power_P90,Power_P95
WaterMonth,,
1.0,22.254572,20.438784
2.0,26.757603,23.782359
3.0,43.750099,39.583353
4.0,77.530930,70.484770
5.0,112.089459,98.372482
6.0,96.524279,0.000000
7.0,96.142607,0.000000
8.0,121.866515,107.814075
9.0,84.580828,66.566983


In [31]:
calmonthly['PropYearsMinAvailFlowGt1UnitRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>rated_flow_unit).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYearsMinAvailFlowGt2UnitRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>rated_flow_unit*2).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYearsMinAvailFlowGt3UnitRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>rated_flow_unit*3).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYearsMinAvailFlowGtCanalRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>canal_capacity).groupby('WaterMonth').count()/len(yearly.index)
calmonthly.fillna(0)
calmonthly

,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,PotentialPower3u_P95,PotentialPower3u_P50,PotentialPower4u_P95,PotentialPower4u_P50,Energy_stdev,Energy_coefvar,PropYearsMinAvailFlowGt1UnitRatedFlow,PropYearsMinAvailFlowGt2UnitRatedFlow,PropYearsMinAvailFlowGt3UnitRatedFlow,PropYearsMinAvailFlowGtCanalRatedFlow
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1.0,Oct,10,0.493289,0.395903,0.399534,0.269664,120.0,93.734194,97.000000,52.0,...,168.257019,170.537797,220.560318,223.950380,6883.756576,0.293074,0.03,NaN,NaN,NaN
2.0,Nov,11,0.405300,0.333046,0.339519,0.210883,114.0,89.626667,93.000000,50.0,...,166.655351,169.774916,218.209197,222.811503,9063.109685,0.310262,0.06,NaN,NaN,NaN
3.0,Dec,12,0.285853,0.239748,0.241942,0.151971,126.0,100.371290,104.000000,59.0,...,160.040351,166.804103,208.676408,218.433073,15178.690753,0.286356,0.26,0.03,NaN,NaN
4.0,Jan,1,0.220992,0.172098,0.175889,0.096964,150.0,118.035484,120.000000,73.0,...,146.913513,161.265967,190.270558,210.419160,20094.599541,0.232588,0.97,0.25,0.05,NaN
5.0,Feb,2,0.184513,0.126249,0.135856,0.023904,197.0,139.093227,137.000000,85.0,...,107.653122,153.270340,136.537153,199.086104,17967.206051,0.179736,1.00,0.88,0.38,0.12
6.0,Mar,3,0.174545,0.095402,0.091399,0.040419,315.0,194.305161,193.258065,107.0,...,47.119083,123.268080,31.168967,157.667615,30547.192785,0.294403,1.00,0.98,0.80,0.56
7.0,Apr,4,0.186841,0.086423,0.083453,0.050048,364.0,231.410000,217.000000,125.0,...,74.575318,111.397736,70.958952,141.558395,28602.906345,0.294217,1.00,1.00,0.95,0.81
8.0,May,5,0.213956,0.130269,0.127393,0.065320,340.0,265.630968,276.000000,147.0,...,96.557313,125.027017,120.485237,160.071169,17276.307038,0.151620,1.00,0.95,0.89,0.72
9.0,Jun,6,0.367320,0.195159,0.190663,0.108489,293.0,213.166667,210.000000,119.0,...,129.361485,148.968499,166.000358,193.077750,22953.167588,0.214589,0.95,0.63,0.28,0.06


In [32]:
calmonthly['PropYears1uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower1u']].where(lambda x: x['PotentialPower1u']>rated_power_unit).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYears2uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower2u']].where(lambda x: x['PotentialPower2u']>rated_power_unit*2).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYears3uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower3u']].where(lambda x: x['PotentialPower3u']>rated_power_unit*3).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYears4uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower4u']].where(lambda x: x['PotentialPower4u']>yearly['Power_max'].max()).groupby('WaterMonth').count()/len(yearly.index)

calmonthly.fillna(0)
calmonthly


,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,Energy_stdev,Energy_coefvar,PropYearsMinAvailFlowGt1UnitRatedFlow,PropYearsMinAvailFlowGt2UnitRatedFlow,PropYearsMinAvailFlowGt3UnitRatedFlow,PropYearsMinAvailFlowGtCanalRatedFlow,PropYears1uPotentialGtRatedPower,PropYears2uPotentialGtRatedPower,PropYears3uPotentialGtRatedPower,PropYears4uPotentialGtRatedPower
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1.0,Oct,10,0.493289,0.395903,0.399534,0.269664,120.0,93.734194,97.000000,52.0,...,6883.756576,0.293074,0.03,NaN,NaN,NaN,1.00,1.00,1.00,1.00
2.0,Nov,11,0.405300,0.333046,0.339519,0.210883,114.0,89.626667,93.000000,50.0,...,9063.109685,0.310262,0.06,NaN,NaN,NaN,1.00,1.00,1.00,1.00
3.0,Dec,12,0.285853,0.239748,0.241942,0.151971,126.0,100.371290,104.000000,59.0,...,15178.690753,0.286356,0.26,0.03,NaN,NaN,1.00,1.00,1.00,1.00
4.0,Jan,1,0.220992,0.172098,0.175889,0.096964,150.0,118.035484,120.000000,73.0,...,20094.599541,0.232588,0.97,0.25,0.05,NaN,0.96,0.95,0.94,0.91
5.0,Feb,2,0.184513,0.126249,0.135856,0.023904,197.0,139.093227,137.000000,85.0,...,17967.206051,0.179736,1.00,0.88,0.38,0.12,0.74,0.72,0.69,0.62
6.0,Mar,3,0.174545,0.095402,0.091399,0.040419,315.0,194.305161,193.258065,107.0,...,30547.192785,0.294403,1.00,0.98,0.80,0.56,0.31,0.30,0.29,0.21
7.0,Apr,4,0.186841,0.086423,0.083453,0.050048,364.0,231.410000,217.000000,125.0,...,28602.906345,0.294217,1.00,1.00,0.95,0.81,0.16,0.14,0.12,0.08
8.0,May,5,0.213956,0.130269,0.127393,0.065320,340.0,265.630968,276.000000,147.0,...,17276.307038,0.151620,1.00,0.95,0.89,0.72,0.20,0.17,0.14,0.12
9.0,Jun,6,0.367320,0.195159,0.190663,0.108489,293.0,213.166667,210.000000,119.0,...,22953.167588,0.214589,0.95,0.63,0.28,0.06,0.67,0.63,0.56,0.45


## Flow duration curve

In [33]:
daily['ExceedanceRnd']=np.round(daily['Exceedance']*2,2)/2
fdc=daily.reset_index().groupby('ExceedanceRnd').mean(numeric_only=True).drop(['VicFalls','Conversion','Year','Month','Day','MonthId','WaterYear','WaterDay','WaterMonth','WaterWeek'],axis=1)
fdc

,Flow,Exceedance,Volume,Flow_difference,Flow_difference_pct,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec,PotentialPower1u,PotentialPower2u,PotentialPower3u,PotentialPower4u
ExceedanceRnd,,,,,,,,,,,,,,,,,,,,,
0.000,8772.477705,0.001419,0.757942,141.768831,0.016565,0.027432,1.094595,66.351351,12.472973,14.959459,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.005,6359.324763,0.004984,0.549446,104.868507,0.016113,0.045385,1.109890,72.164835,13.824176,17.626374,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.994568,16.113375,12.304689,0.000000
0.010,5561.165271,0.010051,0.480485,69.253341,0.012486,0.083112,1.311224,73.091837,13.576531,17.668367,...,2.090979,2.090979,2.090979,8.363918,200.734020,0.001555,24.288687,47.464585,68.360146,22.654002
0.015,5151.846959,0.015138,0.445120,96.759711,0.018854,0.104023,1.408046,70.683908,12.856322,16.390805,...,22.515603,22.515603,22.515603,90.062413,2161.497904,0.017496,26.027919,50.935354,73.542974,90.999867
0.020,4783.656585,0.020092,0.413308,93.399818,0.019504,0.109081,1.394595,69.605405,13.194595,16.610811,...,24.281242,24.281242,24.281242,97.124967,2330.999198,0.020321,27.814913,54.504942,78.882388,98.065300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.980,181.465971,0.980187,0.015679,2.731835,0.015036,0.890341,4.431818,28.545455,4.346591,2.750000,...,NaN,NaN,NaN,24.838461,596.123054,0.136871,57.650768,114.853614,171.060917,224.735657
0.985,172.968359,0.985248,0.014944,1.936288,0.011162,0.907573,4.514563,27.689320,4.237864,2.582524,...,NaN,NaN,NaN,23.492392,563.817409,0.135852,57.631373,114.812932,170.995159,224.636685
0.990,162.994547,0.990457,0.014083,1.974990,0.012095,0.943537,4.823171,25.896341,4.341463,2.439024,...,NaN,NaN,NaN,22.353063,536.473509,0.137091,57.608500,114.764971,170.917669,224.520128


## Add overall summary data to the model file

In [34]:
autocorrel=pd.DataFrame(statools.acf(yearly['Energy'],fft=True,qstat=False,alpha=0.05))
autocorrel

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,1.0,0.434225,0.285721,0.223134,0.215102,0.159847,0.121485,0.085341,0.169592,0.096722,...,-0.038707,-0.109022,-0.039256,-0.166306,-0.149395,-0.227741,-0.232639,-0.257802,-0.204877,-0.145523
1,"[1.0, 1.0]","[0.23822820546022994, 0.6302210023682407]","[0.05571963700037444, 0.5157231446367607]","[-0.020120454077178246, 0.46638910676425804]","[-0.03589181377695941, 0.4660967538180246]","[-0.09813200994985671, 0.4178251403678589]","[-0.140270542548405, 0.38324068644276066]","[-0.17857121302755352, 0.3492541162148626]","[-0.09537820100077998, 0.4345631315639158]","[-0.17238659245853188, 0.36582966281180945]",...,"[-0.3095873543587424, 0.23217424325547067]","[-0.3801150757074277, 0.16207128501368823]","[-0.31202866181164235, 0.23351578785209526]","[-0.43929543490004186, 0.10668290024891514]","[-0.4262486524025247, 0.12745888647791853]","[-0.5076747191155357, 0.05219221886798281]","[-0.5196014534651454, 0.0543238902585807]","[-0.551919900784924, 0.03631689419087042]","[-0.5075516044731351, 0.09779729621897892]","[-0.45347841397150657, 0.16243294507344308]"


In [35]:
lowheadshutdowns=daily.loc[daily['LowHeadShutoff']==1].groupby('WaterYear').count()['Flow'].to_frame().rename(columns={"Flow":"Days"})

lowheadshutdowns['StartDate']=daily[['WaterYear','Flow']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').min()['Date']
lowheadshutdowns['EndDate']=daily[['WaterYear','Flow']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').max()['Date']
lowheadshutdowns['MonthId']=daily[['WaterYear','MonthId']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').min()['MonthId']

#daily.loc[daily['LowHeadShutoff']==1].groupby('WaterYear').min()#['Date']
lowheadshutdowns['Month_power_max']=monthly.loc[lowheadshutdowns['MonthId']][['WaterYear','Power_max']].set_index('WaterYear')
lowheadshutdowns

,Days,StartDate,EndDate,MonthId,Month_power_max
WaterYear,,,,,
1933,7,1934-03-24,1934-03-30,1934.03,182.799202
1947,42,1948-03-03,1948-04-13,1948.03,103.582240
1951,22,1952-03-01,1952-03-22,1952.03,102.410808
1952,13,1953-03-21,1953-04-02,1953.03,189.747973
1955,17,1956-05-03,1956-05-19,1956.05,112.448214
1956,27,1957-03-02,1957-03-28,1957.03,103.236353
1957,61,1958-02-03,1958-04-04,1958.02,107.262023
1960,24,1961-04-01,1961-04-24,1961.04,96.901522
1961,17,1962-04-02,1962-04-18,1962.04,101.424990


In [36]:
lowflowshutdowns=daily.loc[daily['LowFlowShutoff']==1].groupby('Year').count()['Flow'].to_frame().rename(columns={"Flow":"Days"})

lowflowshutdowns['StartDate']=daily[['Year','Flow']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').min()['Date']
lowflowshutdowns['EndDate']=daily[['Year','Flow']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').max()['Date']
lowflowshutdowns['MonthId']=daily[['Year','MonthId']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').min()['MonthId']


lowflowshutdowns['Month_power_max']=monthly.loc[lowflowshutdowns['MonthId']][['Year','Power_max']].set_index('Year')
lowflowshutdowns

,Days,StartDate,EndDate,MonthId,Month_power_max
Year,,,,,
1924,10,1924-10-01,1924-10-10,1924.1,18.041268


In [37]:

models.at[model_name,'Mean']=yearly['Energy'].mean()
models.at[model_name,'Max']=yearly['Energy'].max()
models.at[model_name,'Min']=yearly['Energy'].min()

models.at[model_name,'P95']=yearly['Energy'].quantile(0.05)
models.at[model_name,'P90']=yearly['Energy'].quantile(0.10)
models.at[model_name,'P75']=yearly['Energy'].quantile(0.25)
models.at[model_name,'P50']=yearly['Energy'].quantile(0.5)
models.at[model_name,'P25']=yearly['Energy'].quantile(0.75)
models.at[model_name,'P10']=yearly['Energy'].quantile(0.90)
models.at[model_name,'P5']=yearly['Energy'].quantile(0.95)

models.at[model_name,'StDev']=yearly['Energy'].std()
models.at[model_name,'CoefVar']=models.at[model_name,'StDev']/models.at[model_name,'Mean']

models.at[model_name,'CapFactor']=yearly['CapFactor'].mean()

models.at[model_name,'CalYearMean']=calyearly['Energy'].mean()
models.at[model_name,'CalYearStDev']=calyearly['Energy'].std()
models.at[model_name,'CalYearCoefVar']=models.at[model_name,'CalYearStDev']/models.at[model_name,'CalYearMean']

models.at[model_name,'Power_max']=yearly['Power_max'].max()
models.at[model_name,'Power_Turbine1_max']=yearly['PowerTurbine1_max'].max()

models.at[model_name,'CapFactor_Power_max']=calyearly['Energy'].mean()/(yearly['Power_max'].max()*24*365)

models.at[model_name,'RatedPower']=rated_power_unit

models.at[model_name,'P95FirmPower']=power_exceed.iloc[19]
models.at[model_name,'P90FirmPower']=power_exceed.iloc[18]

models.loc[model_name]



C:\Users\Andy Fleming\AppData\Local\Temp\ipykernel_54632\3956752596.py:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  models.at[model_name,'P95FirmPower']=power_exceed.iloc[19]
C:\Users\Andy Fleming\AppData\Local\Temp\ipykernel_54632\3956752596.py:30: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  models.at[model_name,'P90FirmPower']=power_exceed.iloc[18]


OutputPrefix                            base
Description                        Base Case
EWRCatSet                   Recommendation 1
TailwaterLift                            0.0
HeadpondLift                             0.0
Capacity                                 180
MinimumHead                               10
MinimumFlowUnit                           50
FixedTailwater                             0
UnitsAvailable                             4
RatedDischarge                           250
RatedHead                               21.7
RatedTurbineEfficiency                 0.943
RatedGeneratorEfficiency                0.98
MaximumLoadFactor                        1.1
OptimiseTurbines                           0
RatedPower                         49.182038
Mean                           842358.878765
P95                            704627.099528
P90                            757469.585218
P75                            802552.139264
P50                            847227.819699
P25       

## Save Files

In [38]:
outp=output_data+out_prefix

if dryrun==False:
    daily.to_csv(outp+'_pe_daily.csv')
    monthly.to_csv(outp+'_pe_monthly.csv')
    yearly.to_csv(outp+'_pe_yearly.csv')
    edc.to_csv(outp+'_pe_percentiles.csv')
    monthly_day.to_csv(outp+'_pe_monthly_day.csv')
    daily_slim.to_csv(outp+'_pe_daily_slim.csv')
    weekly.to_csv(outp+'_pe_weekly.csv')
    weekly_slim.to_csv(outp+'_pe_weekly_slim.csv')
    calmonthly.to_csv(outp+'_pe_calmonthly.csv')
    power_exceed.to_csv(outp+'_pe_power_exceed.csv')
    fdc.to_csv(outp+'_pe_fdc.csv')
    autocorrel.to_csv(outp+'_pe_autocorrel.csv')
    lowheadshutdowns.to_csv(outp + "lowheadshutdowns.csv")
    lowflowshutdowns.to_csv(outp + "lowflowshutdowns.csv")

    if year != '':
        models.to_csv(input_data + year + "/models.csv")
    else:
        models.to_csv(input_data + "models.csv")

In [41]:
if output_separate_years:
    years=daily['WaterYear'].unique().tolist()
    for year in years:
        days=daily.loc[daily.WaterYear==year]
        days.to_csv(output_data + 'years/' + out_prefix + '_pe_daily_' + str(year) + '.csv')
 